МФТИ ФИВТ, Машинное обучение, Никита Волков

# Ранжирование

* Дедлайн **14 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями.
* Кагл: https://inclass.kaggle.com/c/ml-mipt-ranking4
* Инвайт: https://kaggle.com/join/4uthnui9sndi

### Небольшая предыстория

Внезапно очутившись на 4 курсе (как это произошло, вообще не понятно) студенты ФИВТа поняли, что времени стало совсем не хватать. Немного подумав, они решили, что во всем виноваты обезумевшие преподаватели курса Машинное обучение. Было проведено коллективное собрание, на котором было принято решение — попробовать договориться с преподавателями. Однако, все попытки примирения были отвергнуты. И тогда на очередном собрании (вместо выполнения очередного ДЗ ;)) было принято решение сделать вопросно-ответную систему для студентов. Например, она сможет четко отвечать на вопросы следующего типа ”Когда дедлайн по очередному огромному заданию?”, ”Что нужно сделать в задании?”, а вопрос ”Сколько суток нужно потратить, чтобы выполнить очередное ДЗ” даже никто и задавать не будет — никто их не делает, кроме самих преподавателей. А на вопрос ”Сможем ли мы переубедить Арсения Ашуху?” система ответит картинкой:

<img width=400 src="./pic.png">

Для каждого вопроса студенты смогли написать систему, которая подбирает наиболее подходящие ответы в интернете. А вот, как из упорядочить, они не знают, поскольку прогуляли лекцию по ранжированию. К счастью, было записано видео этой лекции, так что вы сможете им помочь.

А теперь серьезно...

### Формат файлов

Вам выдается 4 файла:

* `train.txt` --- обучающая выборка пар запрос-документ и асессорские метки релевантности
* `test.txt` --- тестовая выборка пар запрос-документ
* `queries_test.txt` --- запросы из `train.txt`
* `queries_train.txt` --- запросы из `test.txt`

Колонки в файлах могут быть следующего типа:

* `QueryId` --- уникальный номер запроса
* `DocumentId` --- номер документа, не повторяется для одного запроса
* `DocumentLink` --- url документа
* `Relevance` --- асессорская метка релевантности

Формат файла ответов приведен ниже. Пары запрос-документ должны соответсвовать файлу `test.txt` и должны быть упорядочены по убыванию построенной функции релевантности. То есть так, как в поисковой выдаче.

Качество оценивается с помощью метрики ${NDCG}_{10}$.

### Бейзлайн

In [1]:
from bs4 import BeautifulSoup
from urllib import urlopen
import numpy as np

Считываем все запросы из test

In [2]:
with open('queries_test.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

По очереди загружаем все документы, считая для каждого TF и сортируя для каждого запроса. Это и есть скор в бейзлайне.

In [3]:
def dcg (y, i):
    #print np.log2(float(i + 2))
    return float(2 ** y - 1) / np.log2(float(i + 2))

def ndcg(y_true, y_pred, k = 5):
    score = 0.
    max_score = 0.
    for j in range(np.min([k, y_pred.shape[0], len(y_true)])):
        #print dcg(y_pred[j], j)
        score += dcg(y_pred[j], j)
        max_score += dcg(y_true[j], j)
    return score/max_score

In [4]:
def predict(filename = 'test.txt', accuracy = False, k = 2, b = 0.75):
    with open(filename) as ft:
        with open('baseline-3.txt', 'w') as fb:
            ft.readline()
            fb.write('QueryId,DocumentId\n')
            numberOfDoc = 0.
            numberOfDocForQuery = 0.
            numbersOfDocForQuery = []
            textSizes = 0.
            qid_prev = 0
            acc = []
            counters = {}
            qids, docids, relevs, scores = [], [], [], []
            for line in ft:
                numberOfDoc += 1
                #print line
                try:
                    if accuracy:
                        relev, qid, docid, url = line.split(',')
                    else:
                        qid, docid, url = line.split(',')
                except Exception:
                    continue
                qid, docid = int(qid), int(docid)
                if qid != qid_prev:
                    numbersOfDocForQuery.append(numberOfDocForQuery)
                    numberOfDocForQuery = 1
                    counters = {}
                    #print qid
                #else:
                #    numberOfDocForQuery += 1
                #if len(qids) > 0 and qid != qids[-1]:
                    if qid_prev != 0:
                        order = np.argsort(relevs)[::-1]
                        qids = np.array(qids)[order]
                        docids = np.array(docids)[order]
                        if accuracy:
                            acc.append(ndcg(scores, np.array(scores)[order]))
                        else:
                            for i in range(len(qids)):
                                fb.write('{},{}\n'.format(qids[i], docids[i]))
                        
                    qids, docids, relevs, scores = [], [], [], []
                else:
                    numberOfDocForQuery += 1
                try:
                    text = ' '.join(BeautifulSoup(urlopen(url), 'lxml').stripped_strings)
                except Exception:
                    text = 0
                    continue
                text = np.array([t.encode('utf-8') for t in text.split(' ')])
                textSizes += text.size
                frec = 0
                for word in queries[qid - 101].split(' '):
                    if qid != qid_prev:
                        counters[word] = (text == word).mean() > 0
                    else:
                        counters[word] += (text == word).mean() > 0
                    try:
                        frec += ((text == word).mean() * (k + 1)) / ((text == word).mean() + k*(1 - b - text.size * numberOfDoc / textSizes)) * np.log(float(numberOfDocForQuery) / float(counters[word]))
                    except ZeroDivisionError:
                        frec += (text == word).mean()
                if accuracy:
                    scores.append(float(relev))
                qid_prev = qid
                qids.append(qid)
                docids.append(docid)
                relevs.append(frec)
                #print relevs
    if accuracy:
        print np.mean(acc)

In [5]:
for k in range (2,5):
    print 'k = ', k, predict('train.txt', accuracy= True, k = k, b = 0.75)

k =  2 0.326218251899
None
k =  3 0.325847562884
None
k =  4

KeyboardInterrupt: 

Если у вас не загружаются некоторые страницы, подумайте, как с этим можно бороться. Например, посмотрите код с семинара и из предыдущего ДЗ. Если и это не помогает, то нужно подумать, что делать в этой ситуации.

### Кажется, это последнее задание

Свобода!

<img width=200 src="./last.jpg">

А, нет, еще экзамен...